# L3: Multi-agent Customer Support Automation

In this lesson, you will learn about the six key elements which help make Agents perform even better:
- Role Playing
- Focus
- Tools
- Cooperation
- Guardrails
- Memory

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, API and LLM

In [2]:
from crewai import Agent, Task, Crew
from crewai import LLM
from dotenv import load_dotenv
import os

load_dotenv()  # this will load env keys from .env file

True

In [3]:
#api_key = os.getenv('GROQ_API_KEY')
llm = LLM(
    model="ollama/llama3.2",
    #model='groq/gemma2-9b-it',
    temperature = 0.7
)

## Role Playing, Focus and Cooperation

In [4]:
support_agent = Agent(
    role="Senior Support Representative",
	goal="Be the most friendly and helpful "
        "support representative in your team",
	backstory=(
		"You work at crewAI (https://crewai.com) and "
        " are now working on providing "
		"support to {customer}, a super important customer "
        " for your company."
		"You need to make sure that you provide the best support!"
		"Make sure to provide full complete answers, "
        " and make no assumptions."
	),
	allow_delegation=False,
	verbose=True,
	llm=llm
)

- By not setting `allow_delegation=False`, `allow_delegation` takes its default value of being `True`.
- This means the agent _can_ delegate its work to another agent which is better suited to do a particular task. 

In [5]:
support_quality_assurance_agent = Agent(
	role="Support Quality Assurance Specialist",
	goal="Get recognition for providing the "
    "best support quality assurance in your team",
	backstory=(
		"You work at crewAI (https://crewai.com) and "
        "are now working with your team "
		"on a request from {customer} ensuring that "
        "the support representative is "
		"providing the best support possible.\n"
		"You need to make sure that the support representative "
        "is providing full"
		"complete answers, and make no assumptions."
	),
	verbose=True,
	llm=llm
)

* **Role Playing**: Both agents have been given a role, goal and backstory.
* **Focus**: Both agents have been prompted to get into the character of the roles they are playing.
* **Cooperation**: Support Quality Assurance Agent can delegate work back to the Support Agent, allowing for these agents to work together.

## Tools, Guardrails and Memory

### Tools

- Import CrewAI tools

In [6]:
from crewai_tools import SerperDevTool, \
                         ScrapeWebsiteTool, \
                         WebsiteSearchTool

### Possible Custom Tools
- Load customer data
- Tap into previous conversations
- Load data from a CRM
- Checking existing bug reports
- Checking existing feature requests
- Checking ongoing tickets
- ... and more

- Some ways of using CrewAI tools.

```Python
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
```

- Instantiate a document scraper tool.
- The tool will scrape a page (only 1 URL) of the CrewAI documentation.

In [7]:
docs_scrape_tool = ScrapeWebsiteTool(
    #website_url="https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/",
    website_url="https://jadooai.com/",
    )

##### Different Ways to Give Agents Tools

- Agent Level: The Agent can use the Tool(s) on any Task it performs.
- Task Level: The Agent will only use the Tool(s) when performing that specific Task.

**Note**: Task Tools override the Agent Tools.

### Creating Tasks
- You are passing the Tool on the Task Level.

In [8]:
inquiry_resolution = Task(
    description=(
        "{customer} just reached out with a super important task:\n"
	    "{inquiry}\n\n"
        "{person} from {customer} is the one that reached out. "
		"Make sure to use everything you know "
        "to provide the best support possible."
		"You must strive to provide a complete "
        "and accurate response to the customer's inquiry."
    ),
    expected_output=(
	    "A detailed, informative response to the "
        "customer's inquiry that addresses "
        "all aspects of their question.\n"
        "The response should include references "
        "to everything you used to find the answer, "
        "including external data or solutions. "
        "Ensure the answer is complete, "
		"leaving no questions unanswered, and maintain a helpful and friendly "
		"tone throughout."
    ),
	tools=[docs_scrape_tool],
    agent=support_agent,
)

- `quality_assurance_review` is not using any Tool(s)
- Here the QA Agent will only review the work of the Support Agent

In [9]:
quality_assurance_review = Task(
    description=(
        "Review the response drafted by the Senior Support Representative for {customer}'s inquiry. "
        "Ensure that the answer is comprehensive, accurate, and adheres to the "
		"high-quality standards expected for customer support.\n"
        "Verify that all parts of the customer's inquiry "
        "have been addressed "
		"thoroughly, with a helpful and friendly tone.\n"
        "Check for references and sources used to "
        " find the information, "
		"ensuring the response is well-supported and "
        "leaves no questions unanswered."
    ),
    expected_output=(
        "A final, detailed, and informative response "
        "ready to be sent to the customer.\n"
        "This response should fully address the "
        "customer's inquiry, incorporating all "
		"relevant feedback and improvements.\n"
		"Don't be too formal, we are a chill and cool company "
	    "but maintain a professional and friendly tone throughout."
    ),
    agent=support_quality_assurance_agent,
)

In [10]:
#pip install langchain-embeddings
embedder={
        "provider": "ollama",
        "config": {"model": "nomic-embed-text"},
 }

### Creating the Crew

#### Memory
- Setting `memory=True` when putting the crew together enables Memory.

In [11]:
crew = Crew(
  agents=[support_agent, support_quality_assurance_agent],
  tasks=[inquiry_resolution, quality_assurance_review],
  verbose=False,
  memory=True,
  llm=llm,
  embedder=embedder
)

### Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

#### Guardrails
- By running the execution below, you can see that the agents and the responses are within the scope of what we expect from them.

In [ ]:
inputs = {
    "customer": "TensorDot Solution",
    "person": "Tariq Jamil",
    "inquiry": "I need help with arranging a training course "
               "on NLP during next month. Which subjects would be covered?"
               "Can you provide guidance?"
}
result = crew.kickoff(inputs=inputs)

2025-04-22 12:03:20,327 - 140424444920256 - rag_storage.py-rag_storage:138 - ERROR: Error during short_term search: Expected Embedings to be non-empty list or numpy array, got [] in query.
2025-04-22 12:03:20,329 - 140424444920256 - rag_storage.py-rag_storage:138 - ERROR: Error during entities search: Expected Embedings to be non-empty list or numpy array, got [] in query.


# Agent: Senior Support Representative
## Task: TensorDot Solution just reached out with a super important task:
I need help with arranging a training course on NLP during next month. Which subjects would be covered?Can you provide guidance?

Tariq Jamil from TensorDot Solution is the one that reached out. Make sure to use everything you know to provide the best support possible.You must strive to provide a complete and accurate response to the customer's inquiry.


# Agent: Senior Support Representative
## Thought: Action: Read website content
## Using tool: Read website content
## Tool Input: 
"{}"
## Tool Output: 

AI School by JadooAI
0
Skip to Content
Home
About Us
Contact
Enroll
Free Workshop
Open Menu
Close Menu
Home
About Us
Contact
Enroll
Free Workshop
Open Menu
Close Menu
Home
About Us
Contact
Enroll
Free Workshop
Master Large Language Models in 10 weeks Transform your AI career with our intensive LLM bootcamp. Gain hands-on expertise, learn from industry experts, and build r

2025-04-22 12:05:58,003 - 140424444920256 - rag_storage.py-rag_storage:109 - ERROR: Error during short_term save: Expected Embedings to be non-empty list or numpy array, got [] in add.




# Agent: Senior Support Representative
## Final Answer: 
The provided text appears to be a course syllabus for an NLP (Natural Language Processing) bootcamp, specifically focusing on Large Language Models (LLMs). The course covers various topics in NLP, including text preprocessing, deep learning, and LLMs. Here's a summary of the key points:

**Course Structure:**

* 10 weeks
* Hands-on exercises and projects

**Topics Covered:**

Week 1-2: Text Preprocessing and Tokenization
* Removing unwanted characters, symbols, HTML tags
* Stopword removal, stemming, lemmatization
* Word vs. sentence tokenization, whitespace, rule-based tokenization (NLTK, spaCy)

Week 3-4: Deep Learning and Transformers
* Limitations of traditional NLP
* The rise of deep learning and hierarchical representations
* Transformer architecture (self-attention, positional encoding)
* Key transformer models (BERT, BART, FLAN)

Week 5-6: Advanced NLP with Transformers
* HuggingFace transformers library introduction an

/home/tjamil/Desktop/AIS-NLP10S/session-09/.venv/lib/python3.12/site-packages/httpx/_content.py:204: DeprecationWarning: Use 'content=<...>' to upload raw bytes/text content.
  warnings.warn(message, DeprecationWarning)
/home/tjamil/Desktop/AIS-NLP10S/session-09/.venv/lib/python3.12/site-packages/httpx/_content.py:204: DeprecationWarning: Use 'content=<...>' to upload raw bytes/text content.
  warnings.warn(message, DeprecationWarning)
/home/tjamil/Desktop/AIS-NLP10S/session-09/.venv/lib/python3.12/site-packages/httpx/_content.py:204: DeprecationWarning: Use 'content=<...>' to upload raw bytes/text content.
  warnings.warn(message, DeprecationWarning)
/home/tjamil/Desktop/AIS-NLP10S/session-09/.venv/lib/python3.12/site-packages/httpx/_content.py:204: DeprecationWarning: Use 'content=<...>' to upload raw bytes/text content.
  warnings.warn(message, DeprecationWarning)
/home/tjamil/Desktop/AIS-NLP10S/session-09/.venv/lib/python3.12/site-packages/httpx/_content.py:204: DeprecationWarning:

Failed to add to long term memory: Failed to convert text into a Pydantic model due to error: 3 validation errors for TaskEvaluation
suggestions
  Input should be a valid array [type=list_type, input_value='["To provide a more deta... or the field of NLP."]', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/list_type
quality
  Input should be a valid number [type=float_type, input_value='8', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/float_type
entities
  Input should be a valid array [type=list_type, input_value='[{"name": "Course Struct...Course Structure\']»}}', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/list_type
# Agent: Support Quality Assurance Specialist
## Task: Review the response drafted by the Senior Support Representative for TensorDot Solution's inquiry. Ensure that the answer is comprehensive, accurate, and adheres to the high-quality standards expected

2025-04-22 12:19:34,668 - 140424444920256 - rag_storage.py-rag_storage:109 - ERROR: Error during short_term save: Expected Embedings to be non-empty list or numpy array, got [] in add.




# Agent: Support Quality Assurance Specialist
## Final Answer: 
Welcome to the NLP Bootcamp with Large Language Models (LLMs) course! In this 10-week comprehensive program, you'll delve into the fundamentals of Natural Language Processing (NLP), explore advanced topics like deep learning and transformer architectures, and learn how to deploy LLM-powered apps using LangChain.

**Course Structure:**
The bootcamp is divided into ten weeks, each focusing on a specific topic. You'll have ample opportunities to engage with hands-on exercises and projects that will help you develop practical skills in NLP and machine learning.

Here's an overview of the course structure:

Week 1-2: **Text Preprocessing and Tokenization**

* Learn how to remove unwanted characters, symbols, HTML tags from text data
* Understand stopwords removal, stemming, lemmatization, and word vs. sentence tokenization
* Familiarize yourself with tools like NLTK and spaCy for efficient tokenization

Week 3-4: **Deep Learn

/home/tjamil/Desktop/AIS-NLP10S/session-09/.venv/lib/python3.12/site-packages/httpx/_content.py:204: DeprecationWarning: Use 'content=<...>' to upload raw bytes/text content.
  warnings.warn(message, DeprecationWarning)
/home/tjamil/Desktop/AIS-NLP10S/session-09/.venv/lib/python3.12/site-packages/httpx/_content.py:204: DeprecationWarning: Use 'content=<...>' to upload raw bytes/text content.
  warnings.warn(message, DeprecationWarning)
/home/tjamil/Desktop/AIS-NLP10S/session-09/.venv/lib/python3.12/site-packages/httpx/_content.py:204: DeprecationWarning: Use 'content=<...>' to upload raw bytes/text content.
  warnings.warn(message, DeprecationWarning)
/home/tjamil/Desktop/AIS-NLP10S/session-09/.venv/lib/python3.12/site-packages/httpx/_content.py:204: DeprecationWarning: Use 'content=<...>' to upload raw bytes/text content.
  warnings.warn(message, DeprecationWarning)
/home/tjamil/Desktop/AIS-NLP10S/session-09/.venv/lib/python3.12/site-packages/httpx/_content.py:204: DeprecationWarning:

- Display the final result as Markdown.

In [ ]:
from IPython.display import Markdown
Markdown(result.raw)